In [97]:
using LinearAlgebra
using Random
using TimerOutputs

In [98]:
# Function to project onto the PSD matrix
function Project_onto_PSD(A_symm::Matrix{Float64})::Matrix{Float64}
    λ, V = eigen(Symmetric(A_symm))  # Eigenvalue decomposition
    λ[λ .< 0] .= 0.0  # Set all negative eigenvalues exactly to zero
    return V * Diagonal(λ) * V'  # Reconstruct PSD matrix
end

Project_onto_PSD (generic function with 1 method)

In [99]:
# Function to project onto the unit diagonal (in-place modification)
function project_onto_UD!(A_symm::Matrix{Float64})::Matrix{Float64}
    for i in 1:size(A_symm,1)
        A_symm[i,i] = 1.0
    end
    return A_symm
end

project_onto_UD! (generic function with 1 method)

In [100]:
# Function for POCS with Dykstra's Correction
function nearest_corr_dykstra(A_symm::Matrix{Float64}, tol::Float64=1e-8, max_iter::Int=1000000)
    Y_k = copy(A_symm)
    ΔS_k = zeros(size(A_symm))

    for k in 1:max_iter
        R_k = Y_k - ΔS_k
        X_k = Project_onto_PSD(R_k)
        ΔS_k = X_k - R_k
        Y_k = project_onto_UD!(copy(X_k))  # Ensure no in-place modification to X_k

        # Convergence check (relative error)
        if norm(Y_k - X_k, Inf) / norm(Y_k, Inf) <= tol
            println("Converged after $k iterations")
            return Y_k, k
        end
    end
    println("Reached maximum iterations ($max_iter) without convergence")
    return Y_k, max_iter
end

nearest_corr_dykstra (generic function with 3 methods)

In [101]:
# ===== Example Usage ===== #
# Pass in the symmetric matrix (not necessarily a valid correlation matrix)

A = [ 2 -1  0  0;
     -1  2 -1  0;
      0 -1  2 -1;
      0  0 -1  2.0]


A_symm = (A + A') / 2  # Make it symmetric

# Run the nearest correlation matrix function
to = TimerOutput()  # Initialize timing
println("=== POCS with Dykstra ===")

@timeit to "POCS with Dykstra" begin
    result, iterations = nearest_corr_dykstra(A_symm)
end

show(to)

# Verify the result is PSD
min_eigenval = minimum(eigen(Symmetric(result)).values)
println("")
println("Minimum Eigenvalue after projection: ", min_eigenval)
result

=== POCS with Dykstra ===
Converged after 19 iterations
──────────────────────────────────────────────────────────────────────────────
                                     Time                    Allocations      
                            ───────────────────────   ────────────────────────
     Tot / % measured:           349ms /  98.6%           11.9MiB /  99.8%    

Section             ncalls     time    %tot     avg     alloc    %tot      avg
──────────────────────────────────────────────────────────────────────────────
POCS with Dykstra        1    344ms  100.0%   344ms   11.8MiB  100.0%  11.8MiB
──────────────────────────────────────────────────────────────────────────────
Minimum Eigenvalue after projection: -5.120313906149992e-9


4×4 Matrix{Float64}:
  1.0       -0.808412   0.191588   0.106775
 -0.808412   1.0       -0.656233   0.191588
  0.191588  -0.656233   1.0       -0.808412
  0.106775   0.191588  -0.808412   1.0

In [102]:
rank(result)

4

In [103]:
# Checking Frobenius norm
frobenius_norm = norm(A-result)  
println(frobenius_norm)

2.133729106546587


In [104]:
eigvals(result)

4-element Vector{Float64}:
 -5.1203141182831794e-9
  0.204427369422255
  1.4505423553260148
  2.3450302803720415

In [105]:
A = [ 2 -1  0  0;
     -1  2 -1  0;
      0 -1  2 -1;
      0  0 -1  2.0]

4×4 Matrix{Float64}:
  2.0  -1.0   0.0   0.0
 -1.0   2.0  -1.0   0.0
  0.0  -1.0   2.0  -1.0
  0.0   0.0  -1.0   2.0

In [106]:
using NearestCorrelationMatrix
online = nearest_cor(A, AlternatingProjections())

4×4 Matrix{Float64}:
  1.0       -0.808412   0.191588   0.106775
 -0.808412   1.0       -0.656233   0.191588
  0.191588  -0.656233   1.0       -0.808412
  0.106775   0.191588  -0.808412   1.0

In [107]:
# Checking Frobenius norm
frobenius_norm = norm(A-online)  
println(frobenius_norm)

2.133729108708922


In [108]:
eigvals(online)

4-element Vector{Float64}:
 1.190868213218337e-16
 0.20442736920963595
 1.4505423539829718
 2.345030276807393

In [109]:
online2 = nearest_cor(A, Newton())

4×4 Symmetric{Float64, Matrix{Float64}}:
  1.0       -0.808412   0.191588   0.106775
 -0.808412   1.0       -0.656233   0.191588
  0.191588  -0.656233   1.0       -0.808413
  0.106775   0.191588  -0.808413   1.0

In [110]:
# Checking Frobenius norm
frobenius_norm = norm(A-online2)  
println(frobenius_norm)

2.133729149891114


In [111]:
eigvals(online2)

4-element Vector{Float64}:
 9.751760805732305e-8
 0.20442733860482118
 1.450542327687486
 2.3450302361900848

In [112]:
online3 = nearest_cor(A, DirectProjection())

4×4 Matrix{Float64}:
  1.0          -0.5           1.71771e-16  -2.25514e-16
 -0.5           1.0          -0.5          -2.10562e-16
  1.71771e-16  -0.5           1.0          -0.5
 -2.25514e-16  -2.10562e-16  -0.5           1.0

In [113]:
# Checking Frobenius norm
frobenius_norm = norm(A-online3)  
println(frobenius_norm)

2.345207879911716


In [114]:
eigvals(online3)

4-element Vector{Float64}:
 0.19098300562505457
 0.6909830056250528
 1.3090169943749472
 1.8090169943749452

In [115]:
issymmetric(online3)

true

In [116]:
isposdef(online3)

true

### Because a correlation matrix is defined as only positive semi-definite, it is possible for an algorithm to converge successfully but still not be useable for other methods such as a cholesky decomposition (common in probability models). Furthermore, the smallest eigenvalues may be negative on the order of machine precision (e.g. -2.2e-16) due to the inherent nature of floating point numbers. If a positive definite matrix is absolutely required, then pass the argument ensure_pd=true to the solver. This will replace the smallest eigenvalues with a small positive value and reconstruct the NCM:

- λ, P = eigen(X)
replace(x -> max(x, ϵ), λ)
X .= P * Diagonal(λ) * P'
cov2cor!(X) # ensure that the transformed matrix is still a correlation matrix
